## Sternberg sample numbers are integers starting at 1.

The purpose this script is to fix the sample numbers so that they are
integers starting at 1.  This script also handles some particular issues
with the Sternberg including the presence of unwanted events.json files
and unwanted .DS_store files.

In [1]:
import os
import datetime
from hed.tools import BidsTsvDictionary, HedLogger
from hed.util import get_file_list, get_new_dataframe, get_value_dict

# Set the specific variables for the Sternberg dataset.
bids_root_path = 'G:\Sternberg\SternbergWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
sampling_rate_file = os.path.realpath(os.path.join(bids_root_path, 'code/samplingRates.tsv'))
log_name = 'sternberg_11_fix_miscellaneous_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Create the file dictionaries
json_files = get_file_list(bids_root_path, extensions=[".json"], exclude_dirs = exclude_dirs, name_suffix='_events')
event_files = get_file_list(bids_root_path, extensions=[".tsv"], exclude_dirs = exclude_dirs, name_suffix='_events')

# Get the sampling rates
sampling_dict = get_value_dict(sampling_rate_file)

# Remove the events.json files
for filename in json_files:
    basename = os.path.basename(filename)
    os.remove(filename)
    logger.add(basename, f"Removed {os.path.basename(filename)}")

# Update the samples
for filename in event_files:
    df = get_new_dataframe(filename)
    basename = os.path.basename(filename)
    sampling_key = basename[:-11].replace("WorkingMemory", "Experiment")
    sampling_rate = sampling_dict[sampling_key]
    for index, value in df['onset'].iteritems():
        df.loc[index, 'sample'] = round(value*sampling_rate) + 1
    df =df.astype({"sample": int})
    logger.add(basename, "Made sample numbers start at 1")
    logger.add(basename, f"Saved the file to {os.path.basename(filename)}")
    df.to_csv(filename, sep='\t', index=False)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
warning_string = "\n\nWARNING Summary:\n" + logger.get_log_string(level="WARNING")
print(log_string)
print(error_string)
print(warning_string)
save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)
    fp.write(warning_string)



Log output:
sternberg_11_fix_miscellaneous_log: Level None
task-WorkingMemory_events.json:
	[ Removed task-WorkingMemory_events.json]
sub-001_ses-01_task-WorkingMemory_run-1_events.json:
	[ Removed sub-001_ses-01_task-WorkingMemory_run-1_events.json]
sub-001_ses-01_task-WorkingMemory_run-2_events.json:
	[ Removed sub-001_ses-01_task-WorkingMemory_run-2_events.json]
sub-001_ses-01_task-WorkingMemory_run-3_events.json:
	[ Removed sub-001_ses-01_task-WorkingMemory_run-3_events.json]
sub-001_ses-01_task-WorkingMemory_run-4_events.json:
	[ Removed sub-001_ses-01_task-WorkingMemory_run-4_events.json]
sub-002_ses-01_task-WorkingMemory_run-1_events.json:
	[ Removed sub-002_ses-01_task-WorkingMemory_run-1_events.json]
sub-002_ses-01_task-WorkingMemory_run-2_events.json:
	[ Removed sub-002_ses-01_task-WorkingMemory_run-2_events.json]
sub-002_ses-01_task-WorkingMemory_run-3_events.json:
	[ Removed sub-002_ses-01_task-WorkingMemory_run-3_events.json]
sub-002_ses-01_task-WorkingMemory_run-4_event